installed dbplyr", version = "2.3.4" because of error in the last version

In [1]:
setwd('/home/thlamp/tcga/data')
library(TCGAbiolinks)
library(SummarizedExperiment)
library(sesameData)
library(sesame)
library(dbplyr)
library(ggplot2)
library(survminer)
library(pheatmap)
library(dplyr)
library(biomaRt)
library(DESeq2)
library(impute)
library(tidyverse)

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges, rowRanks,
    rowSdDiffs, rowSds, rowSums2, ro

# Find cases per sample type and project

In [3]:
projects <- c('TCGA-ACC','TCGA-BLCA','TCGA-BRCA','TCGA-CESC','TCGA-CHOL','TCGA-COAD','TCGA-DLBC','TCGA-ESCA','TCGA-GBM','TCGA-HNSC','TCGA-KICH','TCGA-KIRC','TCGA-KIRP','TCGA-LAML',
              'TCGA-LGG','TCGA-LIHC','TCGA-LUAD','TCGA-LUSC','TCGA-MESO','TCGA-OV','TCGA-PAAD','TCGA-PCPG','TCGA-PRAD','TCGA-READ','TCGA-SARC','TCGA-SKCM','TCGA-STAD','TCGA-TGCT',
              'TCGA-THCA','TCGA-THYM','TCGA-UCEC','TCGA-UCS','TCGA-UVM')

In [ ]:
query <- GDCquery(project = projects[x], data.category = "Transcriptome Profiling",data.type = "Gene Expression Quantification", 
    workflow.type = "STAR - Counts", access = "open", sample.type = c("Primary Tumor","Solid Tissue Normal"))
query <- GDCquery(project = projects[x], data.category = "Transcriptome Profiling",data.type = "Gene Expression Quantification", 
    workflow.type = "STAR - Counts", access = "open", sample.type = c("Primary Tumor","Solid Tissue Normal"))

query.miRNA <- GDCquery(project = "TCGA-BLCA",experimental.strategy = "miRNA-Seq",data.category = "Transcriptome Profiling", 
    data.type = "miRNA Expression Quantification",barcode = c(dataSmTP.miR, dataSmNT.miR), access = "open")

In [32]:
#mrna
project <- c()
TP <- c()
NT <- c()

for (x in 1:length(projects)) {
  query <- GDCquery(project = projects[x], data.category = "Transcriptome Profiling",data.type = "Gene Expression Quantification", 
    workflow.type = "STAR - Counts", access = "open", sample.type = c("Primary Tumor","Solid Tissue Normal"))

  samplesDown <- getResults(query,cols=c("cases"))
  dataSmTP <- TCGAquery_SampleTypes(barcode = samplesDown,typesample = "TP")
  dataSmNT <- TCGAquery_SampleTypes(barcode = samplesDown,typesample = "NT")

  project[x] <- projects[x]
  TP[x] <- length(dataSmTP)
  NT[x] <- length(dataSmNT)

}
projectsDf <- data.frame(projects = project, TPnum = TP, NTnum = NT)
projectsDf

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-ACC

--------------------

oo Filtering results

--------------------

ooo By access

ooo By data.type

ooo By workflow.type

ooo By sample.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-BLCA

--------------------

oo Filtering 

In [4]:
#mirna
project <- c()
TP <- c()
NT <- c()

for (x in 1:length(projects)) {
  query <- GDCquery(project = projects[x], data.category = "Transcriptome Profiling",data.type = "miRNA Expression Quantification", 
    experimental.strategy = "miRNA-Seq", access = "open", sample.type = c("Primary Tumor","Solid Tissue Normal"))

  samplesDown <- getResults(query,cols=c("cases"))
  dataSmTP <- TCGAquery_SampleTypes(barcode = samplesDown,typesample = "TP")
  dataSmNT <- TCGAquery_SampleTypes(barcode = samplesDown,typesample = "NT")

  project[x] <- projects[x]
  TP[x] <- length(dataSmTP)
  NT[x] <- length(dataSmNT)

}
projectsDf <- data.frame(projects = project, TPnum = TP, NTnum = NT)
projectsDf

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-ACC

--------------------

oo Filtering results

--------------------

ooo By access

ooo By experimental.strategy

ooo By data.type

ooo By sample.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-BLCA

--------------------

oo Fi

projects,TPnum,NTnum
<chr>,<int>,<int>
TCGA-ACC,80,0
TCGA-BLCA,417,19
TCGA-BRCA,1096,104
TCGA-CESC,307,3
TCGA-CHOL,36,9
TCGA-COAD,455,8
TCGA-DLBC,47,0
TCGA-ESCA,186,13
TCGA-GBM,8,5


In [5]:
#methylation
project <- c()
TP <- c()
NT <- c()

for (x in 1:length(projects)) {
  query <- GDCquery(project = projects[x], data.category = "DNA Methylation",data.type = "Methylation Beta Value", 
    platform = "Illumina Human Methylation 450", access = "open", sample.type = c("Primary Tumor","Solid Tissue Normal"))

  samplesDown <- getResults(query,cols=c("cases"))
  dataSmTP <- TCGAquery_SampleTypes(barcode = samplesDown,typesample = "TP")
  dataSmNT <- TCGAquery_SampleTypes(barcode = samplesDown,typesample = "NT")

  project[x] <- projects[x]
  TP[x] <- length(dataSmTP)
  NT[x] <- length(dataSmNT)

}
projectsDf <- data.frame(projects = project, TPnum = TP, NTnum = NT)
projectsDf

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-ACC

--------------------

oo Filtering results

--------------------

ooo By platform

ooo By access

ooo By data.type

ooo By sample.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-BLCA

--------------------

oo Filtering resul

projects,TPnum,NTnum
<chr>,<int>,<int>
TCGA-ACC,80,0
TCGA-BLCA,418,21
TCGA-BRCA,793,97
TCGA-CESC,307,3
TCGA-CHOL,36,9
TCGA-COAD,312,38
TCGA-DLBC,48,0
TCGA-ESCA,185,16
TCGA-GBM,140,2


In [54]:
head(query$results[[1]], 2)

,id,data_format,cases,access,file_name,submitter_id,data_category,type,platform,file_size,⋯,analysis_id,analysis_state,analysis_submitter_id,analysis_workflow_link,analysis_workflow_type,analysis_workflow_version,sample_type,is_ffpe,cases.submitter_id,sample.submitter_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>
1,e523c375-f64a-4904-af5c-3da6d2d4f59a,TSV,TCGA-V4-A9EV-01A-11R-A405-07,open,d48466f5-2fbe-455a-9e19-2277f71983d5.rna_seq.augmented_star_gene_counts.tsv,c7503cae-90fe-4a34-9d16-6f358c3233d0,Transcriptome Profiling,gene_expression,Illumina,4231350,⋯,9d897d62-6adb-48f0-8a0d-d3e71c1076a5,released,d48466f5-2fbe-455a-9e19-2277f71983d5_star__counts,https://github.com/NCI-GDC/gdc-rnaseq-cwl/blob/5d8c131bbff59fb0c969217fc1d44e6d1503cd1f/rnaseq-star-align/star2pass.rnaseq_harmonization.cwl,STAR - Counts,5d8c131bbff59fb0c969217fc1d44e6d1503cd1f,Primary Tumor,NA,TCGA-V4-A9EV,TCGA-V4-A9EV-01A
2,1d970f75-011a-4fc6-a151-1ad62e016c35,TSV,TCGA-YZ-A983-01A-11R-A405-07,open,0c792398-8789-4708-bc3a-73b0bde80175.rna_seq.augmented_star_gene_counts.tsv,70c1d0b8-96f9-4947-a5b2-9e745c296a35,Transcriptome Profiling,gene_expression,Illumina,4215959,⋯,6a612a20-87b0-41ea-9867-4ce172e6c5f3,released,0c792398-8789-4708-bc3a-73b0bde80175_star__counts,https://github.com/NCI-GDC/gdc-rnaseq-cwl/blob/5d8c131bbff59fb0c969217fc1d44e6d1503cd1f/rnaseq-star-align/star2pass.rnaseq_harmonization.cwl,STAR - Counts,5d8c131bbff59fb0c969217fc1d44e6d1503cd1f,Primary Tumor,NA,TCGA-YZ-A983,TCGA-YZ-A983-01A


In [33]:
projectsDf <- data.frame(projects = project, TPnum = TP, NTnum = NT)
projectsDf

projects,TPnum,NTnum
<chr>,<int>,<int>
TCGA-ACC,79,0
TCGA-BLCA,412,19
TCGA-BRCA,1111,113
TCGA-CESC,304,3
TCGA-CHOL,35,9
TCGA-COAD,481,41
TCGA-DLBC,48,0
TCGA-ESCA,184,13
TCGA-GBM,158,5


In [41]:
write.csv(projectsDf, "projects.csv")

# Data download and pre-processing

## mRNA

In [3]:
# Download only these cases that are about TP ορ ΝΤ

query <- GDCquery(project = "TCGA-LIHC",data.category = "Transcriptome Profiling",data.type = "Gene Expression Quantification", 
    workflow.type = "STAR - Counts", access = "open", sample.type = c("Primary Tumor","Solid Tissue Normal"))

GDCdownload(query = query)

dataPrep <- GDCprepare(query = query)

saveRDS(dataPrep, file = "downloaded_dataPrep.rds")

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-LIHC

--------------------

oo Filtering results

--------------------

ooo By access

ooo By data.type

ooo By workflow.type

ooo By sample.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------


GDCdownload will download 421 files. A total of 1.774736464 GB

The total size of files is big. We will download files in chunks




Downloading: 230 MB     

|====================================================|100%                      Completed after 23 s 


Starting to add information to samples

 => Add clinical information to samples

 => Adding TCGA molecular information from marker papers

 => Information will have prefix 'paper_' 

Available assays in SummarizedExperiment : 
  => unstranded
  => stranded_first
  => stranded_second
  => tpm_unstrand
  => fpkm_unstrand
  => fpkm_uq_unstrand



In [4]:
dataPrep <- readRDS("downloaded_dataPrep.rds")

In [4]:
dataPrep

class: RangedSummarizedExperiment 
dim: 60660 421 
metadata(1): data_release
assays(6): unstranded stranded_first ... fpkm_unstrand fpkm_uq_unstrand
rownames(60660): ENSG00000000003.15 ENSG00000000005.6 ...
  ENSG00000288674.1 ENSG00000288675.1
rowData names(10): source type ... hgnc_id havana_gene
colnames(421): TCGA-FV-A3I0-01A-11R-A22L-07
  TCGA-DD-A3A6-11A-11R-A22L-07 ... TCGA-CC-A5UD-01A-11R-A28V-07
  TCGA-BC-A10W-11A-11R-A131-07
colData names(64): barcode patient ... released sample.aux

In [6]:
# tp_count <- sum(colData(dataPrep)$shortLetterCode == "TP")
# tp_count
# nt_count <- sum(colData(dataPrep)$shortLetterCode == "NT")
# nt_count

[1] 371

[1] 50

### Raw data

In [ ]:
data_matrix <- TCGAanalyze_Preprocessing(object = dataPrep,datatype = "unstranded") 
# saveRDS(data_matrix,"raw_data_matrix.rds")

Number of outliers: 0



In [ ]:
head(data_matrix, 2)

,TCGA-2F-A9KO-01A-11R-A38B-07,TCGA-2F-A9KP-01A-11R-A38B-07,TCGA-2F-A9KQ-01A-11R-A38B-07,TCGA-2F-A9KR-01A-11R-A38B-07,TCGA-2F-A9KT-01A-11R-A38B-07,TCGA-2F-A9KW-01A-11R-A38B-07,TCGA-4Z-AA7M-01A-11R-A39I-07,TCGA-4Z-AA7N-01A-11R-A39I-07,TCGA-4Z-AA7O-01A-31R-A39I-07,TCGA-4Z-AA7Q-01A-11R-A39I-07,⋯,TCGA-CU-A0YR-11A-13R-A10U-07,TCGA-GC-A3BM-11A-11R-A22U-07,TCGA-GC-A3WC-11A-11R-A22U-07,TCGA-GC-A6I3-11A-11R-A31N-07,TCGA-GD-A2C5-11A-11R-A180-07,TCGA-GD-A3OP-11A-11R-A220-07,TCGA-GD-A3OQ-11A-21R-A220-07,TCGA-K4-A3WV-11A-21R-A22U-07,TCGA-K4-A54R-11A-11R-A26T-07,TCGA-K4-A5RI-11A-11R-A28M-07
ENSG00000000003.15,6074,11615,5414,4368,3313,2648,8417,2536,7920,3733,⋯,1407,4122,2205,3553,13687,10006,10494,9246,4371,3390
ENSG00000000005.6,0,4,1,1,0,0,4,2,0,0,⋯,42,6,12,0,5,3,4,6,11,5


### TPM

In [ ]:
data_matrix <- TCGAanalyze_Preprocessing(object = dataPrep,datatype = "tpm_unstrand") 

Number of outliers: 0



In [ ]:
head(data_matrix, 2)

,TCGA-2F-A9KO-01A-11R-A38B-07,TCGA-2F-A9KP-01A-11R-A38B-07,TCGA-2F-A9KQ-01A-11R-A38B-07,TCGA-2F-A9KR-01A-11R-A38B-07,TCGA-2F-A9KT-01A-11R-A38B-07,TCGA-2F-A9KW-01A-11R-A38B-07,TCGA-4Z-AA7M-01A-11R-A39I-07,TCGA-4Z-AA7N-01A-11R-A39I-07,TCGA-4Z-AA7O-01A-31R-A39I-07,TCGA-4Z-AA7Q-01A-11R-A39I-07,⋯,TCGA-CU-A0YR-11A-13R-A10U-07,TCGA-GC-A3BM-11A-11R-A22U-07,TCGA-GC-A3WC-11A-11R-A22U-07,TCGA-GC-A6I3-11A-11R-A31N-07,TCGA-GD-A2C5-11A-11R-A180-07,TCGA-GD-A3OP-11A-11R-A220-07,TCGA-GD-A3OQ-11A-21R-A220-07,TCGA-K4-A3WV-11A-21R-A22U-07,TCGA-K4-A54R-11A-11R-A26T-07,TCGA-K4-A5RI-11A-11R-A28M-07
ENSG00000000003.15,108.0135,186.7056,72.4201,73.9806,76.9549,49.0722,159.5696,69.2064,155.7212,50.5693,⋯,27.3519,66.9105,39.2798,56.5882,152.7241,105.0533,125.2822,149.4164,77.840,48.6603
ENSG00000000005.6,0.0000,0.1976,0.0411,0.0521,0.0000,0.0000,0.2330,0.1677,0.0000,0.0000,⋯,2.5092,0.2993,0.6569,0.0000,0.1715,0.0968,0.1468,0.2980,0.602,0.2206


In [ ]:
saveRDS(data_matrix, file = "data_matrix.rds")

In [ ]:
data_matrix <- readRDS("data_matrix.rds")

### Remove patient duplicate with the most zeros in the column, for every patient with duplicates

In [ ]:
duplicates_index_to_remove <- function(patient,input_matrix) {
  # for every element in patient search colnames that start with this sample id, whih are from duplicated samples 
  duplicated_columns <- colnames(input_matrix)[grep(patient, colnames(input_matrix))]
    
  # Count zeros in the specified columns
  zero_counts <- colSums(input_matrix[, duplicated_columns] == 0)
  print(zero_counts)

  # find which column for every duplicated sample has more zeros
  index_of_larger_number <- which.max(zero_counts)

  # Find the index from the matrix, of the column with more zeros to remove
  column_index_to_remove <- which(colnames(input_matrix) %in% duplicated_columns[index_of_larger_number])

  return (column_index_to_remove)
}

In [ ]:
# find duplicated samples in colData(dataPrep)
patient_duplicates <- colData(dataPrep)$sample[which(duplicated(colData(dataPrep)$sample))]

# use function duplicates_index_to_remove for every element in patient_duplicates
patient_duplicates_to_remove <- sapply(patient_duplicates, duplicates_index_to_remove, input_matrix = data_matrix)

print("Index of duplicates with the most zeros per patient with duplicates")
patient_duplicates_to_remove

print("The names of the columns that will be removed")
colnames(data_matrix)[patient_duplicates_to_remove]

TCGA-BL-A13J-01A-11R-A10U-07 TCGA-BL-A13J-01A-11R-A277-07 
                       31629                        22987 
TCGA-BL-A13I-01A-11R-A13Y-07 TCGA-BL-A13I-01A-11R-A277-07 
                       31291                        39927 
TCGA-BL-A0C8-01A-11R-A10U-07 TCGA-BL-A0C8-01A-11R-A277-07 
                       29242                        23308 
[1] "Index of duplicates with the most zeros per patient with duplicates"


TCGA-BL-A13J-01A TCGA-BL-A13I-01A TCGA-BL-A0C8-01A 
              31               29               25

[1] "The names of the columns that will be removed"


[1] "TCGA-BL-A13J-01A-11R-A10U-07" "TCGA-BL-A13I-01A-11R-A277-07"
[3] "TCGA-BL-A0C8-01A-11R-A10U-07"

In [ ]:
# Remove the specified-duplicated columns
data_matrix <- data_matrix[, -patient_duplicates_to_remove, drop = FALSE]
dim(data_matrix)

[1] 60660   428

## miRNA


In [5]:
query.miRNA <- GDCquery(project = "TCGA-LIHC",experimental.strategy = "miRNA-Seq",data.category = "Transcriptome Profiling", 
    data.type = "miRNA Expression Quantification", access = "open", sample.type = c("Primary Tumor","Solid Tissue Normal"))

GDCdownload(query = query.miRNA)

dataAssy.miR <- GDCprepare(query = query.miRNA)

rownames(dataAssy.miR) <- dataAssy.miR$miRNA_ID

saveRDS(dataAssy.miR, "downloaded_dataAssy_miR.rds")

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-LIHC

--------------------

oo Filtering results

--------------------

ooo By access

ooo By experimental.strategy

ooo By data.type

ooo By sample.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------


GDCdownload will download 422 files. A total of 21.225213 MB




  |======================================================================| 100%


In [ ]:
# dataAssy.miR <- readRDS("downloaded_dataAssy_miR.rds")

In [6]:
head(dataAssy.miR, 2)

,miRNA_ID,read_count_TCGA-FV-A3I0-01A-11R-A22J-13,reads_per_million_miRNA_mapped_TCGA-FV-A3I0-01A-11R-A22J-13,cross-mapped_TCGA-FV-A3I0-01A-11R-A22J-13,read_count_TCGA-ES-A2HS-01A-11R-A17X-13,reads_per_million_miRNA_mapped_TCGA-ES-A2HS-01A-11R-A17X-13,cross-mapped_TCGA-ES-A2HS-01A-11R-A17X-13,read_count_TCGA-UB-A7MA-01A-11R-A343-13,reads_per_million_miRNA_mapped_TCGA-UB-A7MA-01A-11R-A343-13,cross-mapped_TCGA-UB-A7MA-01A-11R-A343-13,⋯,cross-mapped_TCGA-DD-A3A2-11A-11R-A214-13,read_count_TCGA-DD-A3A2-01A-11R-A214-13,reads_per_million_miRNA_mapped_TCGA-DD-A3A2-01A-11R-A214-13,cross-mapped_TCGA-DD-A3A2-01A-11R-A214-13,read_count_TCGA-ZS-A9CG-01A-11R-A37G-13,reads_per_million_miRNA_mapped_TCGA-ZS-A9CG-01A-11R-A37G-13,cross-mapped_TCGA-ZS-A9CG-01A-11R-A37G-13,read_count_TCGA-ZP-A9D0-01A-11R-A37G-13,reads_per_million_miRNA_mapped_TCGA-ZP-A9D0-01A-11R-A37G-13,cross-mapped_TCGA-ZP-A9D0-01A-11R-A37G-13
,<chr>,<int>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>,<dbl>,<chr>,⋯,<chr>,<int>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>,<dbl>,<chr>
hsa-let-7a-1,hsa-let-7a-1,68731,5039.171,N,16966,6707.596,N,44411,10258.93,N,⋯,N,10481,7425.492,N,31753,9174.288,N,16776,5777.757,N
hsa-let-7a-2,hsa-let-7a-2,68386,5013.876,N,17161,6784.691,N,44620,10307.21,N,⋯,Y,10356,7336.933,Y,31748,9172.843,N,16823,5793.944,Y


### Take columns that start with "read_count_" (for raw data) and create matrix

In [ ]:
read_countData <-  colnames(dataAssy.miR)[grep("read_count_", colnames(dataAssy.miR))]

data_matrix_miR <- dataAssy.miR[,read_countData]

colnames(data_matrix_miR) <- gsub("read_count_","", colnames(data_matrix_miR))

head(data_matrix_miR, 2)

,TCGA-DK-A2I2-01A-11R-A17X-13,TCGA-E7-A6MF-01A-12R-A32J-13,TCGA-DK-A3IV-01A-22R-A21E-13,TCGA-FD-A6TA-01A-12R-A33A-13,TCGA-FD-A62S-01A-11R-A30J-13,TCGA-BT-A20T-01A-11R-A14X-13,TCGA-UY-A78M-01A-21R-A358-13,TCGA-DK-AA6L-01A-11R-A39B-13,TCGA-FD-A5BT-01A-11R-A26X-13,TCGA-KQ-A41S-01A-12R-A33A-13,⋯,TCGA-BT-A2LB-11A-11R-A18B-13,TCGA-GC-A3WC-11A-11R-A22V-13,TCGA-BL-A13J-11A-13R-A10V-13,TCGA-CU-A0YR-11A-13R-A10V-13,TCGA-BT-A2LA-11A-11R-A18B-13,TCGA-BT-A20U-11A-11R-A14X-13,TCGA-K4-A54R-11A-11R-A26X-13,TCGA-GD-A2C5-11A-11R-A17X-13,TCGA-CU-A0YN-11A-11R-A10V-13,TCGA-BT-A20Q-11A-11R-A14X-13
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
hsa-let-7a-1,18587,44934,69295,33473,41076,38391,18145,63904,41545,41340,⋯,57605,39790,51229,29017,86514,8503,36292,102603,25562,21645
hsa-let-7a-2,18617,45179,68297,33315,41241,38219,18167,63826,41446,41514,⋯,57248,39588,51276,28944,86180,8286,35489,101773,26018,21638


In [ ]:
saveRDS(data_matrix_miR,"raw_data_matrix_miR.rds")

### Take columns that start with "reads_per_million_miRNA" (for normalized data) and create matrix

In [ ]:
read_countData <-  colnames(dataAssy.miR)[grep("reads_per_million_miRNA", colnames(dataAssy.miR))]

data_matrix_miR <- dataAssy.miR[,read_countData]

colnames(data_matrix_miR) <- gsub("reads_per_million_miRNA_mapped_","", colnames(data_matrix_miR))

head(data_matrix_miR, 2)

,TCGA-DK-A2I2-01A-11R-A17X-13,TCGA-E7-A6MF-01A-12R-A32J-13,TCGA-DK-A3IV-01A-22R-A21E-13,TCGA-FD-A6TA-01A-12R-A33A-13,TCGA-FD-A62S-01A-11R-A30J-13,TCGA-BT-A20T-01A-11R-A14X-13,TCGA-UY-A78M-01A-21R-A358-13,TCGA-DK-AA6L-01A-11R-A39B-13,TCGA-FD-A5BT-01A-11R-A26X-13,TCGA-KQ-A41S-01A-12R-A33A-13,⋯,TCGA-BT-A2LB-11A-11R-A18B-13,TCGA-GC-A3WC-11A-11R-A22V-13,TCGA-BL-A13J-11A-13R-A10V-13,TCGA-CU-A0YR-11A-13R-A10V-13,TCGA-BT-A2LA-11A-11R-A18B-13,TCGA-BT-A20U-11A-11R-A14X-13,TCGA-K4-A54R-11A-11R-A26X-13,TCGA-GD-A2C5-11A-11R-A17X-13,TCGA-CU-A0YN-11A-11R-A10V-13,TCGA-BT-A20Q-11A-11R-A14X-13
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
hsa-let-7a-1,4227.118,25109.46,9249.861,5408.821,5888.805,11645.36,5980.638,9529.839,6661.821,5619.656,⋯,8035.191,4950.450,3714.047,4619.003,11074.03,5780.253,8489.891,10270.60,4260.987,1261.238
hsa-let-7a-2,4233.940,25246.36,9116.643,5383.290,5912.460,11593.18,5987.890,9518.207,6645.946,5643.309,⋯,7985.394,4925.318,3717.455,4607.383,11031.28,5632.738,8302.043,10187.51,4336.998,1260.830


### Remove patient duplicate with the most zeros in the column, for every patient with duplicates

In [ ]:
# keep first 16 characters from colnames, which are sample ids
sample_ids <- substr(colnames(data_matrix_miR), 0, 16)

# find duplicated samples in colData(dataPrep)
patient_duplicates <- sample_ids[which(duplicated(sample_ids))]

# use function duplicates_index_to_remove for every element in patient_duplicates
patient_duplicates_to_remove <- sapply(patient_duplicates, duplicates_index_to_remove, input_matrix = data_matrix_miR)

print("Index of duplicates with the most zeros per patient with duplicates")
patient_duplicates_to_remove

print("The names of the columns that will be removed")
colnames(data_matrix_miR)[patient_duplicates_to_remove]

TCGA-C4-A0F7-01A-11R-A25G-13 TCGA-C4-A0F7-01A-11R-A085-13 
                        1110                         1318 
TCGA-C4-A0F1-01A-11R-A25G-13 TCGA-C4-A0F1-01A-11R-A035-13 
                        1149                         1309 
TCGA-BL-A0C8-01A-11R-A10V-13 TCGA-BL-A0C8-01A-11R-A27D-13 
                        1215                         1179 
TCGA-BL-A13I-01A-11R-A27D-13 TCGA-BL-A13I-01A-11R-A13X-13 
                        1207                         1274 
TCGA-BL-A13J-01A-11R-A27D-13 TCGA-BL-A13J-01A-11R-A10V-13 
                        1141                         1172 
[1] "Index of duplicates with the most zeros per patient with duplicates"


TCGA-C4-A0F7-01A TCGA-C4-A0F1-01A TCGA-BL-A0C8-01A TCGA-BL-A13I-01A 
             132              148              130              264 
TCGA-BL-A13J-01A 
             375

[1] "The names of the columns that will be removed"


[1] "TCGA-C4-A0F7-01A-11R-A085-13" "TCGA-C4-A0F1-01A-11R-A035-13"
[3] "TCGA-BL-A0C8-01A-11R-A10V-13" "TCGA-BL-A13I-01A-11R-A13X-13"
[5] "TCGA-BL-A13J-01A-11R-A10V-13"

In [ ]:
# Remove the specified-duplicated columns
data_matrix_miR <- data_matrix_miR[, -patient_duplicates_to_remove, drop = FALSE]
dim(data_matrix_miR)

[1] 1881  431

## Methylation


In [7]:
query_met <- GDCquery(project = "TCGA-LIHC", data.category = "DNA Methylation", data.type = "Methylation Beta Value", platform = "Illumina Human Methylation 450", 
    access = "open", sample.type = c("Primary Tumor","Solid Tissue Normal"))

GDCdownload(query = query_met, files.per.chunk = 15)

data_met <- GDCprepare(query_met)

saveRDS(data_met, "downloaded_data_met.rds")

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-LIHC

--------------------

oo Filtering results

--------------------

ooo By platform

ooo By access

ooo By data.type

ooo By sample.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------


GDCdownload will download 427 files. A total of 5.562728787 GB




Downloading: 77 MB      

Downloading: 77 MB      

Downloading: 78 MB      

Downloading: 77 MB        

...

Downloading: 78 MB      

Downloading: 36 MB      

--------------------

oo Reading 427 files

--------------------



|====================================================|100%                      Completed after 48 s 


--------------------

oo Merging 427 files

--------------------

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

Creating a SummarizedExperiment from DNA methylation input

Accessing DNAm annotation from sesame package for: hg38 - HM450

see ?sesameData and browseVignettes('sesameData') for documentation

loading from cache

Starting to add information to samples

 => Add clinical information to samples

 => Adding TCGA molecular information from marker papers

 => Information will have prefix 'paper_' 



In [8]:
data_met <- readRDS("downloaded_data_met.rds")

data_met

class: RangedSummarizedExperiment 
dim: 485577 427 
metadata(1): data_release
assays(1): ''
rownames(485577): cg13869341 cg14008030 ... cg11478607 cg08417382
rowData names(52): address_A address_B ... MASK_extBase MASK_general
colnames(427): TCGA-BC-A110-11A-11D-A132-05
  TCGA-G3-A3CG-01A-11D-A20Z-05 ... TCGA-ZS-A9CG-01A-11D-A36Y-05
  TCGA-ZP-A9D0-01A-11D-A36Y-05
colData names(64): barcode patient ... released sample.aux

In [9]:
meth_matrix <- assay(data_met)
head(meth_matrix, 1)

,TCGA-BC-A110-11A-11D-A132-05,TCGA-G3-A3CG-01A-11D-A20Z-05,TCGA-FV-A3I0-01A-11D-A22H-05,TCGA-G3-AAV5-01A-11D-A36Y-05,TCGA-EP-A3JL-01A-11D-A20Z-05,TCGA-ES-A2HS-11A-11D-A17Z-05,TCGA-DD-A3A6-01A-11D-A22H-05,TCGA-FV-A3I1-01A-11D-A22H-05,TCGA-KR-A7K2-01A-12D-A341-05,TCGA-DD-AACP-01A-11D-A40S-05,⋯,TCGA-CC-A7IF-01A-11D-A33L-05,TCGA-FV-A2QR-11A-11D-A20Z-05,TCGA-DD-A39V-11A-11D-A20Z-05,TCGA-G3-A25V-01A-11D-A16X-05,TCGA-CC-A7IH-01A-11D-A33L-05,TCGA-G3-AAV1-01A-11D-A383-05,TCGA-2Y-A9H9-01A-21D-A39G-05,TCGA-2Y-A9GW-01A-11D-A383-05,TCGA-ZS-A9CG-01A-11D-A36Y-05,TCGA-ZP-A9D0-01A-11D-A36Y-05
cg13869341,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Remove patient duplicate with the most zeros in the column, for every patient with duplicates

In [5]:
duplicates_index_to_remove_for_methylation <- function(patient,input_matrix) {
  # for every element in patient search colnames that start with this sample id, whih are from duplicated samples 
  duplicated_columns <- colnames(input_matrix)[grep(patient, colnames(input_matrix))]
  
  # Count the number of NA values in each specified column
  na_counts <- colSums(is.na(input_matrix[, duplicated_columns]))
  print(na_counts)

  # find which column for every duplicated sample has more zeros
  index_of_larger_number <- which.max(na_counts)

  # Find the index from the matrix, of the column with more zeros to remove
  column_index_to_remove <- which(colnames(input_matrix) %in% duplicated_columns[index_of_larger_number])

  return (column_index_to_remove)
}

In [6]:
# find duplicated samples in colData(dataPrep)
patient_duplicates <- colData(data_met)$sample[which(duplicated(colData(data_met)$sample))]

# use function duplicates_index_to_remove for every element in patient_duplicates
patient_duplicates_to_remove <- sapply(patient_duplicates, duplicates_index_to_remove_for_methylation, input_matrix = meth_matrix)

print("Index of duplicates with the most zeros per patient with duplicates")
patient_duplicates_to_remove

print("The names of the columns that will be removed")
colnames(meth_matrix)[patient_duplicates_to_remove]

[1] "Index of duplicates with the most zeros per patient with duplicates"


named list()

[1] "The names of the columns that will be removed"


ERROR: Error in colnames(meth_matrix)[patient_duplicates_to_remove]: invalid subscript type 'list'


In [ ]:
# Remove the specified-duplicated columns
meth_matrix <- meth_matrix[, -patient_duplicates_to_remove, drop = FALSE]
dim(meth_matrix)

[1] 485577    436

In [12]:
saveRDS(meth_matrix, "processed_meth_matrix.rds")

### Download hg38 manifest by sesame package

In [11]:
meth_matrix <- readRDS("processed_meth_matrix.rds")
dim(meth_matrix)

[1] 433660    427

In [3]:
tsv_path <- sesameAnno_download("HM450.hg38.manifest.gencode.v36.tsv.gz")
df <- sesameAnno_readManifestTSV(tsv_path)
head(df)

Warning message:
“The following named parsers don't match the column names: address_A, address_B, target, nextBase, channel, Probe_ID, mapFlag_A, mapChrm_A, mapPos_A, mapQ_A, mapCigar_A, AlleleA_ProbeSeq, mapNM_A, mapAS_A, mapYD_A, mapFlag_B, mapChrm_B, mapPos_B, mapQ_B, mapCigar_B, AlleleB_ProbeSeq, mapNM_B, mapAS_B, mapYD_B, type”


CpG_chrm,CpG_beg,CpG_end,probe_strand,probeID,genesUniq,geneNames,transcriptTypes,transcriptIDs,distToTSS,CGI,CGIposition
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,15864,15866,-,cg13869341,WASH7P,WASH7P,unprocessed_pseudogene,ENST00000488147.1,13706,NA,NA
chr1,18826,18828,-,cg14008030,MIR6859-1;WASH7P,MIR6859-1;WASH7P,miRNA;unprocessed_pseudogene,ENST00000619216.1;ENST00000488147.1,-1390;10744,NA,NA
chr1,29406,29408,-,cg12045430,MIR1302-2;MIR1302-2HG;WASH7P,MIR1302-2;MIR1302-2HG;MIR1302-2HG;WASH7P,miRNA;lncRNA;lncRNA;unprocessed_pseudogene,ENST00000607096.1;ENST00000469289.1;ENST00000473358.1;ENST00000488147.1,-959;-860;-147;164,CGI:chr1:28735-29737,Island
chr1,29424,29426,-,cg20826792,MIR1302-2;MIR1302-2HG;WASH7P,MIR1302-2;MIR1302-2HG;MIR1302-2HG;WASH7P,miRNA;lncRNA;lncRNA;unprocessed_pseudogene,ENST00000607096.1;ENST00000469289.1;ENST00000473358.1;ENST00000488147.1,-941;-842;-129;146,CGI:chr1:28735-29737,Island
chr1,29434,29436,-,cg00381604,MIR1302-2;MIR1302-2HG;WASH7P,MIR1302-2;MIR1302-2HG;MIR1302-2HG;WASH7P,miRNA;lncRNA;lncRNA;unprocessed_pseudogene,ENST00000607096.1;ENST00000469289.1;ENST00000473358.1;ENST00000488147.1,-931;-832;-119;136,CGI:chr1:28735-29737,Island
chr1,68848,68850,-,cg20253340,OR4F5,OR4F5;OR4F5,protein_coding;protein_coding,ENST00000335137.4;ENST00000641515.2,-206;3430,NA,NA


In [4]:
# keep only probes that exist in matrix
processed_df <- df[which(df$probeID %in% rownames(meth_matrix)), ]
dim(processed_df)

# check for existing chromosomes in annotation dataframe
unique(processed_df$CpG_chrm)

# find the probes that are in chrX or chrY
processed_df$probeID[which(processed_df$CpG_chrm %in% c("chrX", "chrY", NA))]

[1] 433660     12

[1] "chr1"  "chr2"  "chr3"  "chr4"  "chr5"  "chr6"  "chr7"  "chr8"  "chr9" 
[10] "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18"
[19] "chr19" "chr20" "chr21" "chr22" "chrX"  NA

[1] "rs2521373"  "rs798149"   "cg12724894" "rs5926356"  "cg09955705"
 [6] "cg13456241" "cg05965490" "rs5936512"  "rs5987737"  "rs5931272" 
[11] "rs1416770"  "rs6626309"  "cg22940546" "cg01759674" "cg06677538"
[16] "cg06102775" "cg20336016" "cg22666875" "cg09033563" "cg21256200"
[21] "cg05380919" "cg06180910" "cg01744396" "cg10546252" "cg08219170"
[26] "cg16997375" "cg07641578" "cg11598976" "cg01414268" "cg25005368"
[31] "cg01238044" "cg01957799" "cg20064778" "cg24238852" "cg15254640"
[36] "cg24336839" "cg11478607"

In [5]:
# remove the remaining probes from chrX
meth_matrix <- meth_matrix[!rownames(meth_matrix) %in% processed_df$probeID[which(processed_df$CpG_chrm %in% c("chrX", "chrY", NA))], ]
dim(meth_matrix)

[1] 433623    427

In [6]:
# check for the remaining probes from chrX and chrY
which(rownames(meth_matrix) %in% processed_df$probeID[which(processed_df$CpG_chrm %in% c("chrX", "chrY", NA))])

integer(0)

### Keep only the columns that I need

In [7]:
mapping_data <- processed_df[c("probeID","transcriptIDs")]
head(mapping_data)

probeID,transcriptIDs
<chr>,<chr>
cg13869341,ENST00000488147.1
cg24669183,ENST00000357876.6;ENST00000419394.2;ENST00000440196.3;ENST00000634337.2;ENST00000634833.2;ENST00000635509.2;ENST00000641296.1;ENST00000648019.1
cg15560884,ENST00000428504.2;ENST00000506640.2;ENST00000634337.2;ENST00000635509.2
cg01014490,ENST00000506640.2;ENST00000634337.2;ENST00000635509.2;ENST00000412115.2;ENST00000429505.6;ENST00000434264.6;ENST00000443772.2;ENST00000457084.1;ENST00000589899.5;ENST00000609830.1;ENST00000651411.1;ENST00000655765.1;ENST00000656571.1;ENST00000657896.1;ENST00000658648.1;ENST00000665719.1;ENST00000665867.1;ENST00000666217.1;ENST00000669749.1;ENST00000670700.1
cg17505339,ENST00000634337.2;ENST00000635509.2;ENST00000412115.2;ENST00000429505.6;ENST00000443772.2;ENST00000585745.5;ENST00000585768.5;ENST00000586288.1;ENST00000586928.5;ENST00000587530.5;ENST00000588951.5;ENST00000589531.5;ENST00000590848.5;ENST00000591440.5;ENST00000591702.1;ENST00000593022.5;ENST00000651411.1;ENST00000655765.1;ENST00000656571.1;ENST00000657896.1;ENST00000658648.1;ENST00000665719.1;ENST00000665867.1;ENST00000666217.1;ENST00000669749.1;ENST00000670700.1
cg11954957,ENST00000634337.2;ENST00000635509.2


### Retain only probe IDs from the matrix after removing NAs

In [8]:
selected_probe_ids <- rownames(meth_matrix)
mapping_data <- mapping_data[which(mapping_data$probeID %in% selected_probe_ids),]
dim(mapping_data)

[1] 433623      2

### Separate transcriptIDs by ; into different row per id

In [9]:
library(tidyr)
mapping_data <- separate_rows(mapping_data, transcriptIDs, sep=";")
dim(mapping_data)
head(mapping_data,10)

[1] 2535531       2

probeID,transcriptIDs
<chr>,<chr>
cg13869341,ENST00000488147.1
cg24669183,ENST00000357876.6
cg24669183,ENST00000419394.2
cg24669183,ENST00000440196.3
cg24669183,ENST00000634337.2
cg24669183,ENST00000634833.2
cg24669183,ENST00000635509.2
cg24669183,ENST00000641296.1
cg24669183,ENST00000648019.1


### NAs per column

In [ ]:
colSums(is.na(mapping_data))

probeID transcriptIDs 
            0         52475

### Remove probeIDs that don't match with transcriptIDs

In [ ]:
mapping_data <- mapping_data[complete.cases(mapping_data),]
dim(mapping_data)

[1] 2483056       2

### Create column with transcriptIDs_without_version

In [12]:
# mapping_data$transcriptIDs_without_version <-gsub("\\.[0-9]*", "", mapping_data$transcriptIDs)
# head(mapping_data)

### Check values in transcriptIDs_without_version

In [13]:
# test <- grep("*\\..",mapping_data$transcriptIDs)

In [14]:
head(mapping_data)

probeID,transcriptIDs
<chr>,<chr>
cg13869341,ENST00000488147.1
cg24669183,ENST00000357876.6
cg24669183,ENST00000419394.2
cg24669183,ENST00000440196.3
cg24669183,ENST00000634337.2
cg24669183,ENST00000634833.2


### Biomart transcriptIDs to ensemblIDs

In [15]:
library(biomaRt)

In [16]:
ensembl <- useEnsembl(biomart = 'genes',dataset = "hsapiens_gene_ensembl",version = 102)

### Search for unique transcriptIDs to avoid repetitions of same transcriptIDs

In [17]:
# check number of unique transcriptIDs to be sure there are all in bm_results with ensembl_gene_id
length(unique(mapping_data$transcriptIDs))

[1] 175766

In [18]:
searchAttributes(mart = ensembl, pattern = "transcript_id")

,name,description,page
,<chr>,<chr>,<chr>
3,ensembl_transcript_id,Transcript stable ID,feature_page
4,ensembl_transcript_id_version,Transcript stable ID version,feature_page
203,ensembl_transcript_id,Transcript stable ID,structure
204,ensembl_transcript_id_version,Transcript stable ID version,structure
243,ensembl_transcript_id,Transcript stable ID,homologs
244,ensembl_transcript_id_version,Transcript stable ID version,homologs
2922,ensembl_transcript_id,Transcript stable ID,snp
2923,ensembl_transcript_id_version,Transcript stable ID version,snp
2966,ensembl_transcript_id,Transcript stable ID,snp_somatic


In [19]:
# transcript_ids <- unique(mapping_data$transcriptIDs_without_version)
transcript_ids <- unique(mapping_data$transcriptIDs)

# bm_results <- getBM(attributes = c('ensembl_gene_id', 'ensembl_transcript_id'),
#       filters = 'ensembl_transcript_id',
#       values = transcript_ids, 
#       mart = ensembl)
bm_results <- getBM(attributes = c('ensembl_gene_id', 'ensembl_transcript_id_version'),
      filters = 'ensembl_transcript_id_version',
      values = transcript_ids, 
      mart = ensembl)

In [20]:
dim(bm_results)

[1] 175766      2

In [21]:
head(bm_results)

,ensembl_gene_id,ensembl_transcript_id_version
,<chr>,<chr>
1,ENSG00000227232,ENST00000488147.1
2,ENSG00000230021,ENST00000634833.2
3,ENSG00000230021,ENST00000419394.2
4,ENSG00000230021,ENST00000440196.3
5,ENSG00000230021,ENST00000641296.1
6,ENSG00000230021,ENST00000357876.6


In [22]:
head(mapping_data)

probeID,transcriptIDs
<chr>,<chr>
cg13869341,ENST00000488147.1
cg24669183,ENST00000357876.6
cg24669183,ENST00000419394.2
cg24669183,ENST00000440196.3
cg24669183,ENST00000634337.2
cg24669183,ENST00000634833.2


### Match ensembl_gene_id with the data from mapping_data by ensembl_transcript_id

In [23]:
# mapping_data$ensembl_ids <- apply(as.matrix(mapping_data$transcriptIDs_without_version),1, function (x){bm_results$ensembl_gene_id[which(bm_results$ensembl_transcript_id %in% x)]})
# head(mapping_data)
mapping_data$ensembl_ids <- apply(as.matrix(mapping_data$transcriptIDs),1, function (x){bm_results$ensembl_gene_id[which(bm_results$ensembl_transcript_id_version %in% x)]})
head(mapping_data)

probeID,transcriptIDs,ensembl_ids
<chr>,<chr>,<chr>
cg13869341,ENST00000488147.1,ENSG00000227232
cg24669183,ENST00000357876.6,ENSG00000230021
cg24669183,ENST00000419394.2,ENSG00000230021
cg24669183,ENST00000440196.3,ENSG00000230021
cg24669183,ENST00000634337.2,ENSG00000230021
cg24669183,ENST00000634833.2,ENSG00000230021


In [24]:
saveRDS(mapping_data,"mapping_data_ensembl_102.rds")

In [4]:
mapping_data <- readRDS("mapping_data_ensembl_102.rds")
head(mapping_data)

probeID,transcriptIDs,ensembl_ids
<chr>,<chr>,<chr>
cg13869341,ENST00000488147.1,ENSG00000227232
cg24669183,ENST00000357876.6,ENSG00000230021
cg24669183,ENST00000419394.2,ENSG00000230021
cg24669183,ENST00000440196.3,ENSG00000230021
cg24669183,ENST00000634337.2,ENSG00000230021
cg24669183,ENST00000634833.2,ENSG00000230021


### Keep only probeID and ensembl_ids with unique pairs of probeID and ensembl_ids

In [25]:
mapping_data <- mapping_data[c("probeID","ensembl_ids")]
dim(mapping_data)
mapping_data <- mapping_data[!duplicated(mapping_data[,c('probeID','ensembl_ids')]),]
dim(mapping_data)

[1] 2483056       2

[1] 524827      2

In [26]:
head(mapping_data)

probeID,ensembl_ids
<chr>,<chr>
cg13869341,ENSG00000227232
cg24669183,ENSG00000230021
cg15560884,ENSG00000228327
cg15560884,ENSG00000230021
cg01014490,ENSG00000228327
cg01014490,ENSG00000230021


### Check results from matching

In [27]:
which(duplicated(mapping_data$probeID))

[1]      4      6      7      9     12     13     20     22     24     26
   [11]     28     41     43     45     47     49     51     53     55     56
   [21]     58     59     61     62     64     65     67     68    137    160
   [31]    162    164    166    181    183    185    187    189    233    235
   [41]    237    239    241    243    245    247    249    251    253    255
   [51]    257    259    261    279    281    283    285    287    289    303
   [61]    305    307    309    311    313    315    317    319    321    399
   [71]    401    403    405    407    409    411    422    424    426    428
   [81]    430    432    434    539    541    543    545    547    550    552
   [91]    554    556    557    559    560    562    564    566    568    570
  [101]    572    577    579    581    583    585    587    589    591    593
  [111]    595    597    599    601    603    605    607    609    611    613
  [121]    615    617    619    621    623    625    627    629    631    711
  [131]    713    715    717    719    721    723    725    727    785    787
  [141]    789    791    793    795    797    831    833    850    852    854
  [151]    856    858    860    862    864    866    868    870    872    874
  [161]    876    878    880    882    884    886    888    891    893    895
  [171]    897    899    901    903    906    908    910    915    917    919
  [181]    921    923    925    927    929    931    933    935    978    980
  [191]    982   1050   1052   1054   1056   1058   1060   1062   1064   1066
  [201]   1068   1070   1072   1074   1076   1080   1082   1084   1086   1088
  [211]   1092   1094   1096   1098   1100   1103   1105   1107   1109   1111
  [221]   1113   1115   1117   1131   1133   1135   1137   1139   1141   1143
  [231]   1145   1147   1149   1151   1258   1260   1262   1302   1304   1306
  [241]   1308   1310   1312   1314   1316   1318   1320   1323   1325   1327
  [251]   1329   1331   1333   1336   1338   1340   1342   1344   1346   1348
  [261]   1350   1352   1354   1356   1358   1360   1362   1364   1366   1404
  [271]   1406   1408   1410   1412   1414   1416   1426   1428   1430   1432
  [281]   1434   1436   1438   1455   1457   1459   1460   1462   1463   1465
  [291]   1466   1467   1538   1540   1542   1544   1546   1548   1550   1552
  [301]   1554   1556   1558   1560   1738   1740   1742   1744   1746   1748
  [311]   1750   1752   1754   1756   1923   1925   1927   2002   2004   2006
  [321]   2007   2009   2010   2012   2013   2015   2016   2071   2073   2075
  [331]   2077   2079   2081   2083   2247   2249   2251   2253   2255   2257
  [341]   2263   2265   2267   2269   2271   2273   2275   2277   2317   2319
  [351]   2321   2323   2325   2327   2329   2331   2333   2335   2372   2374
  [361]   2376   2378   2380   2382   2384   2386   2388   2390   2392   2394
  [371]   2396   2398   2400   2402   2425   2427   2429   2431   2433   2435
  [381]   2437   2439   2578   2580   2582   2583   2585   2586   2588   2589
  [391]   2591   2592   2594   2595   2683   2685   2687   2689   2691   2693
  [401]   2695   2697   2699   2701   2703   2705   2707   2762   2764   2800
  [411]   2802   2804   2806   2808   2810   2812   2845   2847   2889   2891
  [421]   2893   2895   2897   2899   2901   2903   2905   2907   2909   2911
  [431]   2913   2915   2917   2919   2921   2923   2925   3028   3030   3032
  [441]   3034   3036   3166   3168   3170   3172   3174   3176   3178   3180
  [451]   3182   3354   3356   3362   3364   3366   3368   3370   3372   3374
  [461]   3376   3378   3380   3382   3384   3386   3388   3390   3392   3394
  [471]   3396   3439   3441   3443   3445   3692   3694   3696   3698   3700
  [481]   3702   3704   3759   3761   3763   3765   3767   3769   3771   3773
  [491]   3775   3777   3779   3781   3783   3863   3865   3867   3869   3871
  [501]   3873   3875   3877   3879   3881   3883   3885   3887   3889   3891
  [511]   3893   3895   38

In [49]:
mapping_data[which(mapping_data$probeID %in% mapping_data$probeID[524826]),]

probeID,ensembl_ids
<chr>,<chr>
cg19565306,ENSG00000079974
cg19565306,ENSG00000184319


### Create an empty dataframe to match mapping_data with values from meth_matrix

In [50]:
meth_df <- data.frame(matrix(NA, nrow = nrow(mapping_data), ncol = ncol(meth_matrix)+ncol(mapping_data)))
dim(meth_df)

[1] 524827    429

### Set colnames and insert probeID and ensembl_ids

In [51]:
colnames(meth_df)[1:2] <- c("probeID", "ensembl_ids") 
meth_df$probeID <- mapping_data$probeID
meth_df$ensembl_ids <- mapping_data$ensembl_ids
colnames(meth_df)[3:ncol(meth_df)] <- colnames(meth_matrix)
head(meth_df)

,probeID,ensembl_ids,TCGA-BC-A110-11A-11D-A132-05,TCGA-G3-A3CG-01A-11D-A20Z-05,TCGA-FV-A3I0-01A-11D-A22H-05,TCGA-G3-AAV5-01A-11D-A36Y-05,TCGA-EP-A3JL-01A-11D-A20Z-05,TCGA-ES-A2HS-11A-11D-A17Z-05,TCGA-DD-A3A6-01A-11D-A22H-05,TCGA-FV-A3I1-01A-11D-A22H-05,⋯,TCGA-CC-A7IF-01A-11D-A33L-05,TCGA-FV-A2QR-11A-11D-A20Z-05,TCGA-DD-A39V-11A-11D-A20Z-05,TCGA-G3-A25V-01A-11D-A16X-05,TCGA-CC-A7IH-01A-11D-A33L-05,TCGA-G3-AAV1-01A-11D-A383-05,TCGA-2Y-A9H9-01A-21D-A39G-05,TCGA-2Y-A9GW-01A-11D-A383-05,TCGA-ZS-A9CG-01A-11D-A36Y-05,TCGA-ZP-A9D0-01A-11D-A36Y-05
,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,cg13869341,ENSG00000227232,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,cg24669183,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,cg15560884,ENSG00000228327,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,cg15560884,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,cg01014490,ENSG00000228327,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,cg01014490,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Insert values to meth_df from meth_matrix matched by probeID

In [52]:
transfer_values <- function(df, mat) {
  matching_rows <- match(df$probeID, rownames(mat))
  df[, 3:ncol(df)] <- mat[matching_rows, ]
  return(df)
}

meth_df_updated <- transfer_values(meth_df, meth_matrix)

In [53]:
head(meth_df_updated,100)

,probeID,ensembl_ids,TCGA-BC-A110-11A-11D-A132-05,TCGA-G3-A3CG-01A-11D-A20Z-05,TCGA-FV-A3I0-01A-11D-A22H-05,TCGA-G3-AAV5-01A-11D-A36Y-05,TCGA-EP-A3JL-01A-11D-A20Z-05,TCGA-ES-A2HS-11A-11D-A17Z-05,TCGA-DD-A3A6-01A-11D-A22H-05,TCGA-FV-A3I1-01A-11D-A22H-05,⋯,TCGA-CC-A7IF-01A-11D-A33L-05,TCGA-FV-A2QR-11A-11D-A20Z-05,TCGA-DD-A39V-11A-11D-A20Z-05,TCGA-G3-A25V-01A-11D-A16X-05,TCGA-CC-A7IH-01A-11D-A33L-05,TCGA-G3-AAV1-01A-11D-A383-05,TCGA-2Y-A9H9-01A-21D-A39G-05,TCGA-2Y-A9GW-01A-11D-A383-05,TCGA-ZS-A9CG-01A-11D-A36Y-05,TCGA-ZP-A9D0-01A-11D-A36Y-05
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,cg13869341,ENSG00000227232,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,cg24669183,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,cg15560884,ENSG00000228327,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,cg15560884,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,cg01014490,ENSG00000228327,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,cg01014490,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,cg01014490,ENSG00000237491,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,cg17505339,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,cg17505339,ENSG00000237491,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
# saveRDS(meth_df_updated,"meth_df_updated_with_na.rds")

In [ ]:
# meth_df_updated <- readRDS("meth_df_updated_with_na.rds")

### Check results from id mapping

In [54]:
head(meth_df_updated,2)

,probeID,ensembl_ids,TCGA-BC-A110-11A-11D-A132-05,TCGA-G3-A3CG-01A-11D-A20Z-05,TCGA-FV-A3I0-01A-11D-A22H-05,TCGA-G3-AAV5-01A-11D-A36Y-05,TCGA-EP-A3JL-01A-11D-A20Z-05,TCGA-ES-A2HS-11A-11D-A17Z-05,TCGA-DD-A3A6-01A-11D-A22H-05,TCGA-FV-A3I1-01A-11D-A22H-05,⋯,TCGA-CC-A7IF-01A-11D-A33L-05,TCGA-FV-A2QR-11A-11D-A20Z-05,TCGA-DD-A39V-11A-11D-A20Z-05,TCGA-G3-A25V-01A-11D-A16X-05,TCGA-CC-A7IH-01A-11D-A33L-05,TCGA-G3-AAV1-01A-11D-A383-05,TCGA-2Y-A9H9-01A-21D-A39G-05,TCGA-2Y-A9GW-01A-11D-A383-05,TCGA-ZS-A9CG-01A-11D-A36Y-05,TCGA-ZP-A9D0-01A-11D-A36Y-05
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,cg13869341,ENSG00000227232,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,cg24669183,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [55]:
head(meth_matrix,2)

,TCGA-BC-A110-11A-11D-A132-05,TCGA-G3-A3CG-01A-11D-A20Z-05,TCGA-FV-A3I0-01A-11D-A22H-05,TCGA-G3-AAV5-01A-11D-A36Y-05,TCGA-EP-A3JL-01A-11D-A20Z-05,TCGA-ES-A2HS-11A-11D-A17Z-05,TCGA-DD-A3A6-01A-11D-A22H-05,TCGA-FV-A3I1-01A-11D-A22H-05,TCGA-KR-A7K2-01A-12D-A341-05,TCGA-DD-AACP-01A-11D-A40S-05,⋯,TCGA-CC-A7IF-01A-11D-A33L-05,TCGA-FV-A2QR-11A-11D-A20Z-05,TCGA-DD-A39V-11A-11D-A20Z-05,TCGA-G3-A25V-01A-11D-A16X-05,TCGA-CC-A7IH-01A-11D-A33L-05,TCGA-G3-AAV1-01A-11D-A383-05,TCGA-2Y-A9H9-01A-21D-A39G-05,TCGA-2Y-A9GW-01A-11D-A383-05,TCGA-ZS-A9CG-01A-11D-A36Y-05,TCGA-ZP-A9D0-01A-11D-A36Y-05
cg13869341,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
cg24669183,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [56]:
# convert df to matrix without two first columns
meth_mat_updated <- as.matrix(meth_df_updated[-c(1,2)])

rownames(meth_mat_updated) <- meth_df_updated$probeID

colnames(meth_mat_updated) <- colnames(meth_df_updated)[3:ncol(meth_df_updated)]

# keep unique rownames, if the one row is ok then the duplicate will be the same
check_meth_mat_updated <- meth_mat_updated[!duplicated(rownames(meth_mat_updated)), ]

# Identify common row names
common_row_names <- intersect(rownames(check_meth_mat_updated), rownames(meth_matrix))

# Check if values for common row names are identical
values_identical <- identical(check_meth_mat_updated[common_row_names, ], meth_matrix[common_row_names, ])

# Print the results
if (values_identical) {
  print("The values for the common row names are identical")
} else {
  print("The values for the common row names are not identical")
}

[1] "The values for the common row names are identical"


### Calculate mean value per column for common ensembl ids

In [57]:
meth_df_updated <- meth_df_updated[,2:length(colnames(meth_df_updated))]
head(meth_df_updated)

,ensembl_ids,TCGA-BC-A110-11A-11D-A132-05,TCGA-G3-A3CG-01A-11D-A20Z-05,TCGA-FV-A3I0-01A-11D-A22H-05,TCGA-G3-AAV5-01A-11D-A36Y-05,TCGA-EP-A3JL-01A-11D-A20Z-05,TCGA-ES-A2HS-11A-11D-A17Z-05,TCGA-DD-A3A6-01A-11D-A22H-05,TCGA-FV-A3I1-01A-11D-A22H-05,TCGA-KR-A7K2-01A-12D-A341-05,⋯,TCGA-CC-A7IF-01A-11D-A33L-05,TCGA-FV-A2QR-11A-11D-A20Z-05,TCGA-DD-A39V-11A-11D-A20Z-05,TCGA-G3-A25V-01A-11D-A16X-05,TCGA-CC-A7IH-01A-11D-A33L-05,TCGA-G3-AAV1-01A-11D-A383-05,TCGA-2Y-A9H9-01A-21D-A39G-05,TCGA-2Y-A9GW-01A-11D-A383-05,TCGA-ZS-A9CG-01A-11D-A36Y-05,TCGA-ZP-A9D0-01A-11D-A36Y-05
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000227232,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,ENSG00000228327,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,ENSG00000228327,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,ENSG00000230021,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [58]:
# group by ensembl_ids and then calculate the mean per column that start with TCGA for every ensembl id
meth_df_updated <- meth_df_updated %>% group_by(ensembl_ids) %>% summarise(across(starts_with("TCGA"), ~mean(., na.rm = TRUE)))
dim(meth_df_updated)
head(meth_df_updated)

[1] 34925   428

### Convert df with mean values per gene to matrix

In [60]:
meth_mat_mean <- as.matrix(meth_df_updated[, 2:ncol(meth_df_updated)])

rownames(meth_mat_mean) <- meth_df_updated$ensembl_ids

head(meth_mat_mean)

,TCGA-BC-A110-11A-11D-A132-05,TCGA-G3-A3CG-01A-11D-A20Z-05,TCGA-FV-A3I0-01A-11D-A22H-05,TCGA-G3-AAV5-01A-11D-A36Y-05,TCGA-EP-A3JL-01A-11D-A20Z-05,TCGA-ES-A2HS-11A-11D-A17Z-05,TCGA-DD-A3A6-01A-11D-A22H-05,TCGA-FV-A3I1-01A-11D-A22H-05,TCGA-KR-A7K2-01A-12D-A341-05,TCGA-DD-AACP-01A-11D-A40S-05,⋯,TCGA-CC-A7IF-01A-11D-A33L-05,TCGA-FV-A2QR-11A-11D-A20Z-05,TCGA-DD-A39V-11A-11D-A20Z-05,TCGA-G3-A25V-01A-11D-A16X-05,TCGA-CC-A7IH-01A-11D-A33L-05,TCGA-G3-AAV1-01A-11D-A383-05,TCGA-2Y-A9H9-01A-21D-A39G-05,TCGA-2Y-A9GW-01A-11D-A383-05,TCGA-ZS-A9CG-01A-11D-A36Y-05,TCGA-ZP-A9D0-01A-11D-A36Y-05
ENSG00000000419,0.1118654,0.1143980,0.1143608,0.1149675,0.1149448,0.1104212,0.1129604,0.1126091,0.1117635,0.1212908,⋯,0.1106292,0.1115512,0.1157110,0.1123388,0.1140919,0.1265432,0.1170645,0.1454730,0.1195181,0.1174418
ENSG00000000457,0.2624041,0.2641697,0.2292393,0.2560104,0.2675130,0.2650047,0.2805592,0.2577086,0.2540554,0.2558886,⋯,0.2531981,0.2686214,0.2657113,0.2764750,0.2498936,0.2651302,0.2590962,0.2732508,0.2638572,0.2524639
ENSG00000000460,0.4408774,0.4649433,0.4739239,0.4283614,0.4270649,0.4356567,0.4332333,0.4365524,0.4559803,0.3921394,⋯,0.4991599,0.4210997,0.4330659,0.4215809,0.4201691,0.4094430,0.2916599,0.4213613,0.4639523,0.4890818
ENSG00000000938,0.5875732,0.6660703,0.7535626,0.6985967,0.6280740,0.6217274,0.5796593,0.6229398,0.7549736,0.5847583,⋯,0.7383103,0.6139359,0.6306827,0.5307470,0.5765935,0.5515960,0.4751344,0.5907317,0.6067352,0.6824947
ENSG00000000971,0.6138440,0.5702442,0.9245516,0.9152154,0.6372258,0.7066659,0.7477711,0.6248396,0.5694234,0.9101840,⋯,0.7242839,0.7076355,0.7113619,0.6706541,0.6278866,0.7355552,0.9435972,0.8582431,0.9104393,0.9003275
ENSG00000001036,0.3373643,0.3585446,0.3049815,0.3426298,0.3415840,0.3579221,0.3540281,0.3316234,0.3600834,0.3075088,⋯,0.3688745,0.3605740,0.3489792,0.3524409,0.3589586,0.3139935,0.3563082,0.3460009,0.3567931,0.3081536


In [61]:
saveRDS(meth_mat_mean,"processed_meth_mat_mean.rds")

In [2]:
meth_mat_mean <- readRDS("processed_meth_mat_mean.rds")
dim(meth_mat_mean)
head(meth_mat_mean, 2)

[1] 34925   427

,TCGA-BC-A110-11A-11D-A132-05,TCGA-G3-A3CG-01A-11D-A20Z-05,TCGA-FV-A3I0-01A-11D-A22H-05,TCGA-G3-AAV5-01A-11D-A36Y-05,TCGA-EP-A3JL-01A-11D-A20Z-05,TCGA-ES-A2HS-11A-11D-A17Z-05,TCGA-DD-A3A6-01A-11D-A22H-05,TCGA-FV-A3I1-01A-11D-A22H-05,TCGA-KR-A7K2-01A-12D-A341-05,TCGA-DD-AACP-01A-11D-A40S-05,⋯,TCGA-CC-A7IF-01A-11D-A33L-05,TCGA-FV-A2QR-11A-11D-A20Z-05,TCGA-DD-A39V-11A-11D-A20Z-05,TCGA-G3-A25V-01A-11D-A16X-05,TCGA-CC-A7IH-01A-11D-A33L-05,TCGA-G3-AAV1-01A-11D-A383-05,TCGA-2Y-A9H9-01A-21D-A39G-05,TCGA-2Y-A9GW-01A-11D-A383-05,TCGA-ZS-A9CG-01A-11D-A36Y-05,TCGA-ZP-A9D0-01A-11D-A36Y-05
ENSG00000000419,0.1118654,0.1143980,0.1143608,0.1149675,0.1149448,0.1104212,0.1129604,0.1126091,0.1117635,0.1212908,⋯,0.1106292,0.1115512,0.1157110,0.1123388,0.1140919,0.1265432,0.1170645,0.1454730,0.1195181,0.1174418
ENSG00000000457,0.2624041,0.2641697,0.2292393,0.2560104,0.2675130,0.2650047,0.2805592,0.2577086,0.2540554,0.2558886,⋯,0.2531981,0.2686214,0.2657113,0.2764750,0.2498936,0.2651302,0.2590962,0.2732508,0.2638572,0.2524639


In [5]:
head(mapping_data)

probeID,transcriptIDs,ensembl_ids
<chr>,<chr>,<chr>
cg13869341,ENST00000488147.1,ENSG00000227232
cg24669183,ENST00000357876.6,ENSG00000230021
cg24669183,ENST00000419394.2,ENSG00000230021
cg24669183,ENST00000440196.3,ENSG00000230021
cg24669183,ENST00000634337.2,ENSG00000230021
cg24669183,ENST00000634833.2,ENSG00000230021


In [6]:
check_and_compare <- function(ensembl_id, mapping_data, meth_matrix, meth_mat_mean) {
  # Subset the mapping_data based on the ensembl_id
  check_subset <- mapping_data[mapping_data$ensembl_ids %in% ensembl_id, ]
  
  # Remove duplicated rows based on 'probeID' and 'ensembl_ids', because use only one time per probe id every ensembl id 
  check_subset <- check_subset[!duplicated(check_subset[, c('probeID', 'ensembl_ids')]), ]
    
  # if length(check_subset$probeID)== 1 then it returns error in colMeans
  if (length(check_subset$probeID)>1) {
      # Calculate the column means for the selected rows in meth_matrix
      matrix_col_means <- colMeans(meth_matrix[check_subset$probeID, ])
      
      # Check if the column means are identical to the corresponding row in meth_mat_mean
      is_identical <- all.equal(meth_mat_mean[ensembl_id, ], matrix_col_means)
  # if length(check_subset$probeID)== 1 I compare the row from meth_mat_mean with the unique row from meth_matrix
  } else {
      is_identical <- all.equal(meth_mat_mean[ensembl_id,],meth_matrix[check_subset$probeID,])
  }
  return(is_identical = is_identical)
}


In [16]:
# vector of Ensembl IDs from rownames
ensembl_ids_vector <- rownames(meth_mat_mean)[30001:34925]

# Apply check_and_compare to each Ensembl ID in the vector
results_list <- lapply(ensembl_ids_vector, function(ensembl_id) {
  check_and_compare(ensembl_id, mapping_data, meth_matrix, meth_mat_mean)
})

# Convert the list to a vector if needed
results_vector <- unlist(results_list)

# Print ensembl_ids that they aren't identical
print("Results: ")
rownames(meth_mat_mean)[which(! rownames(meth_mat_mean)[30001:34925] %in% ensembl_ids_vector)]

[1] "Results: "


character(0)